# Marimekko Charts aus dem Zensus 2011 mit Python

In [22]:
import pandas as pd
import matplotlib.pyplot as plt

In [23]:
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['font.family'] = 'Statis Sans'

## Tutorials, Inspiration
https://towardsdatascience.com/marimekko-charts-with-pythons-matplotlib-6b9784ae73a1  
https://pythonguides.com/stacked-bar-chart-matplotlib/
## Daten
Gebäude mit Wohnraum nach Baujahr und Heizungsart  
https://ergebnisse2011.zensus2022.de/datenbank//online?operation=table&code=3000G-2004

In [24]:
# download flatfile csv from web interface
# beware: leading zeros in regional ID, windows encoding when using web interface
df_ffcsv_web = pd.read_csv("3000G-2004-GEOLK1_flat.csv", sep=";", encoding="1252", na_values=['-'], \
                           dtype={"1_Auspraegung_Code": str, "GEB001__Gebaeude_mit_Wohnraum__Anzahl": float})\
                            .fillna(0)

In [25]:
# shorten labels
df_ffcsv_web = df_ffcsv_web.replace("Einzel-/Mehrraumöfen (auch Nachtspeicherheizung)", "Ofen/ Nachtspeicher")
df_ffcsv_web = df_ffcsv_web.replace("Keine Heizung im Gebäude oder in den Wohnungen", "Keine Heizung")
df_ffcsv_web = df_ffcsv_web.replace("Fernheizung (Fernwärme)", "Fernwärme")

In [26]:
# drop sums
df_ffcsv_web = df_ffcsv_web[df_ffcsv_web["2_Auspraegung_Label"]!="Insgesamt"]
df_ffcsv_web = df_ffcsv_web[df_ffcsv_web["3_Auspraegung_Label"]!="Insgesamt"]

In [27]:
df_ffcsv_web.head(3)

,Statistik_Code,Statistik_Label,Zeit_Code,Zeit_Label,Zeit,1_Merkmal_Code,1_Merkmal_Label,1_Auspraegung_Code,1_Auspraegung_Label,2_Merkmal_Code,2_Merkmal_Label,2_Auspraegung_Code,2_Auspraegung_Label,3_Merkmal_Code,3_Merkmal_Label,3_Auspraegung_Code,3_Auspraegung_Label,GEB001__Gebaeude_mit_Wohnraum__Anzahl
8,3000G,Gebäude,STAG,Stichtag,09.05.2011,GEOLK1,Landkreise und kreisfreie Städte,01001,"Flensburg, Stadt",GEBBJ3,Baujahr (Mikrozensus-Klassen),BAU0000B1918,Vor 1919,HZGAT1,Heizungsart,HEIZ-FERN,Fernwärme,2148.0
9,3000G,Gebäude,STAG,Stichtag,09.05.2011,GEOLK1,Landkreise und kreisfreie Städte,01001,"Flensburg, Stadt",GEBBJ3,Baujahr (Mikrozensus-Klassen),BAU0000B1918,Vor 1919,HZGAT1,Heizungsart,HEIZ-ETAGE,Etagenheizung,5.0
10,3000G,Gebäude,STAG,Stichtag,09.05.2011,GEOLK1,Landkreise und kreisfreie Städte,01001,"Flensburg, Stadt",GEBBJ3,Baujahr (Mikrozensus-Klassen),BAU0000B1918,Vor 1919,HZGAT1,Heizungsart,HEIZ-BLOCK,Blockheizung,3.0


In [28]:
laenderNamenLang = {
        "01" : "Schleswig-Holstein",
        "02" : "Freie und Hansestadt Hamburg",
        "03" : "Niedersachsen",
        "04" : "Freie Hansestadt Bremen",
        "05" : "Nordrhein-Westfalen",
        "06" : "Hessen",
        "07" : "Rheinland-Pfalz",
        "08" : "Baden-Württemberg",
        "09" : "Freistaat Bayern",
        "10" : "Saarland",
        "11" : "Berlin",
        "12" : "Brandenburg",
        "13" : "Mecklenburg-Vorpommern",
        "14" : "Freistaat Sachsen",
        "15" : "Sachsen-Anhalt",
        "16" : "Freistaat Thüringen"
}

In [29]:
laenderNamenAbk = {
        "01" : "SH",
        "02" : "HH",
        "03" : "NI",
        "04" : "HB",
        "05" : "NW",
        "06" : "HE",
        "07" : "RP",
        "08" : "BW",
        "09" : "BY",
        "10" : "SL",
        "11" : "BE",
        "12" : "BB",
        "13" : "MV",
        "14" : "SN",
        "15" : "ST",
        "16" : "TH"
}

In [30]:
columncolor = {
    "Fernwärme" : "darkmagenta",
    "Etagenheizung" : "darkorange",
    "Blockheizung" : "green",
    "Zentralheizung" : "gold",
    "Ofen/ Nachtspeicher" : "steelblue",
    "Keine Heizung" : "silver"
}

In [31]:
heizungSort = ["Fernwärme","Etagenheizung","Blockheizung","Zentralheizung","Ofen/ Nachtspeicher","Keine Heizung"]

In [41]:
def marimekkoChart(chartAGS):
    
    tmp_df = df_ffcsv_web[df_ffcsv_web["1_Auspraegung_Code"]==chartAGS]\
                .rename(columns={"2_Auspraegung_Label": "Baujahr"})
    
    tmp_Name = tmp_df["1_Auspraegung_Label"].iloc[0]

    tmp_df = tmp_df.pivot(index="Baujahr", columns="3_Auspraegung_Label", values="GEB001__Gebaeude_mit_Wohnraum__Anzahl")\
                .reindex(["Vor 1919", "1919 - 1948", "1949 - 1978", "1979 - 1986", "1987 - 1990", 
                "1991 - 1995", "1996 - 2000", "2001 - 2004", "2005 - 2008", "2009 und später"])

    dims = (18,10)
    fig, ax = plt.subplots(1, figsize=dims, dpi=120, facecolor='white')

    plt.title("Gebäude mit Wohnraum nach Baujahr und Heizungsart", fontsize = 18, fontweight="bold", loc="left", pad=75)
    ax.text(0, 1.08, tmp_Name+" ("+laenderNamenAbk[chartAGS[0:2]]+") 2011", ha="left", fontsize = 16)

    x = tmp_df.sum(axis=1)
    x_label = tmp_df.index

    width = [i/sum(x) for i in x]
    # calculate x coordinates based on the width of the previous bars
    # same as: [0, width[0], width[0] + width[1], width[0] + width[1] + width[2]]
    adjusted_x, temp = [0], 0
    for i in width[:-1]:
        temp += i
        adjusted_x.append(temp)

    # Marimekko chart
    bottom_j = 0

    # for j in tmp_df.columns:
    for j in heizungSort:
        
        ant_j = tmp_df[j]/x
        plt.bar(adjusted_x, ant_j, bottom = bottom_j, width=width, align = 'edge', edgecolor='black', linewidth=.5, label=j, color=columncolor[j])
        bottom_j += ant_j
        
    # x and y ticks (%)
    ax.set_yticks([0, 0.25, 0.5, 0.75, 1])
    ax.set_yticklabels(['0%', '25%', '50%', '75%', '100%'], fontsize=10)
    ax.set_xticks([(width[i]/2)+ v for i, v in enumerate(adjusted_x)])
    #ax.set_xticklabels(x_label, fontsize=10)
    # MAKE SURE THESE LABELS MATCH INDEX POSITIONS
    # use only for line breaks and other layout finetuning
    ax.set_xticklabels(["Vor\n1919", "1919\n-\n1948", "1949\n-\n1978", "1979\n-\n1986", "1987\n-\n1990",\
                "1991\n-\n1995", "1996\n-\n2000", "2001\n-\n2004", "2005\n-\n2008", "ab\n2009"], fontsize=10)
    plt.ylim(0,1)
    plt.xlim(0,1)
    # twin y-axis to draw x-ticks at the top
    axy = ax.twiny()
    axy.set_xticks([0, 0.25, 0.5, 0.75, 1])
    axy.set_xticklabels(['0%', '25%', '50%', '75%', '100%'], fontsize=10)
    ax.set_ylabel('Heizungsart', fontsize = 14, labelpad=10)
    ax.set_xlabel('Gebäude nach Baujahr', fontsize = 14, labelpad=10)

    # horizontal legend
    # fig.legend(bbox_to_anchor=[0.45, 0.01], loc='center', ncol=len(tmp_df.columns), frameon=False)
    handles, labels = ax.get_legend_handles_labels()
    plt.legend(handles[::-1], labels[::-1], frameon=False, loc=5, labelspacing = 1.2, fontsize=12, handletextpad=0.7, bbox_to_anchor=(1.18, 0.25))

    plt.savefig('charts/'+chartAGS+'_'+tmp_Name+'.png', bbox_inches='tight', pad_inches=0.25)
    plt.close()
    #    plt.show()

In [42]:
# test chart function once
marimekkoChart("15083")

In [34]:
# all regional IDs (NUTS-3 in this case)
GEOLK1_AGS = df_ffcsv_web["1_Auspraegung_Code"]
GEOLK1_AGS = GEOLK1_AGS.drop_duplicates()

In [43]:
for meinAGS in GEOLK1_AGS:
    try: 
        marimekkoChart(meinAGS)
    except:
        plt.close()
        print(meinAGS+" failed")